# Einführung in Natural Language Processing

Im folgenden werden anhand von Beispielen mithilfe der Software Libraries:
- spacy - eine Toolsammlung um Probleme im Bereich NLP mithilfe von vortrainierten Deeplearning zu lösen
- nltk - eher akademische Sammlung von Tools im Bereich NLP
- TextBlob - abstraktion für einige spezifische deutsche Corpora (z.B. ready to use sentiment analysis…)

einige elementare Methoden im Bereich NLP vorgestellt.

In [78]:
import random

from IPython.display import HTML

import nltk
import spacy
from spacy import displacy
from textblob_de import TextBlobDE as TextBlob

#load spacy de corpora
nlp = spacy.load("de")

Dieses Notebook dient als Ergänzung zu den Slides:

In [53]:
HTML('<iframe src="//lilith.slides.com/lilith/deck-31/embed?token=ivKQJ292" width="576" height="420" scrolling="no" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe>')

## Preprocessing

Für die meisten Tasks werden wir spacy benutzen, da es einerseits das aus meiner Sicht "industry-grade" tool für NLP und außerdem sehr einfach zu bedienen ist. In Spacy können wir das komplette lingustische preprocessing eines Satzes mit einem einzelnen Kommando erledigen:

In [79]:
nlp("Wie wird das Wetter morgen?")

Wie wird das Wetter morgen?

Mit ```nlp("<text>")``` durchläuft der Satz eine ganze Pipeline an funktionen und gibt den komplett annotierten Text als ein document zurück. Die Pipelines sehen dabei für jede Sprache etwas anders aus beinhalten grundsätzlich allerdings immer folgende Koponenten:
- Tokenizer
- POS-Tagging
- Dependency Parsing
- NER-Tagging

Die Pipeline für Deutsch sieht so aus:

In [80]:
nlp.pipeline

[('tagger', <spacy.pipeline.Tagger at 0x7f264a731908>),
 ('parser', <spacy.pipeline.DependencyParser at 0x7f264ace0c50>),
 ('ner', <spacy.pipeline.EntityRecognizer at 0x7f264a3a7150>)]

![Pipeline](https://spacy.io/assets/img/pipeline.svg)
Im Folgenden werden wir uns mit den einzelnen Funktionen innerhalb dieser Pipeline beschäftigen.

### Tokenizing

Um Text sinnvoll weiterverarbeiten zu können, müssen wir ihn erst in seine einzelnen Bestandteile zerlegen. Das können einelne Wörter, Satzteile oder Sätze (Sentence Boundary Detection) sein.

In [60]:
nltk.tokenize.WordPunctTokenizer().tokenize("Wie wird das Wetter morgen in Hamburg?")

['Wie', 'wird', 'das', 'Wetter', 'morgen', 'in', 'Hamburg', '?']

In [59]:
[token.text for token in nlp("Wie wird das Wetter morgen in Hamburg?")]
    

['Wie', 'wird', 'das', 'Wetter', 'morgen', 'in', 'Hamburg', '?']

Augenscheinlich machen sowohl der NLTK als auch der spacy tokenizer hier genau das selbe. Allerdings funktioniert der NLTK Tokenizer im Hintergrund quasi mit einem Regex, während spacy dafür erst den Satz selbst nach Spaces separiert und dann mit einzelnen Regeln je nach Sprache auch auf Edge-Cases wie Fremdwörter im Deutschen, die mit Bindestrich geschrieben werden, rücksicht nimmt. 

![caption](https://spacy.io/assets/img/tokenization.svg)

Auch wenn das jetzt für Sprachen mit Lateinischen Alphabet erstmal relativ einfach aussieht, gibt es einige Sprachen (Chinesisch, Thailändisch, …) in denen das eine deutlich schwierigere Aufgabe ist.

In [33]:
nltk.tokenize.WordPunctTokenizer().tokenize("Davon hatte Dr. Müller kein Back-up gemacht!")

['Davon',
 'hatte',
 'Dr',
 '.',
 'Müller',
 'kein',
 'Back',
 '-',
 'up',
 'gemacht',
 '!']

In [34]:
[token.text for token in nlp("Davon hatte Dr. Müller kein Back-up gemacht!")]


['Davon', 'hatte', 'Dr.', 'Müller', 'kein', 'Back-up', 'gemacht', '!']

Einen guten Tokenizer braucht man normalerweise nicht nur bei fast allen Anwendungsfällen im Bereich Natural Language Processing, sondern auch vielen weiteren Bereichen wie z.B. Suchsystemen. Außerdem sollte man bedenken, das fehler in diesem Bereich problematisch für alle weiteren Arbeitsschritte sein können.

Beliebte weitere Fallstricke:
- Abkürzungen wie Dr., Fr., … sollten als ein Token gesehen werden und nicht zu einer Satztrennung führen
- Zahlen, Telefonnummern, feststehende Begriffe, … (je nach Level auf dem man tokenized vgl. High-Level-Tokenization)
- Abkürzungen wie I'm, You're, …

(https://www.ibm.com/developerworks/community/blogs/nlp/entry/tokenization?lang=en)

### Part of Speech tagging (POS)

Beim POS tagging geht es darum einen Satz in seine grammatikalischen Satzteile zu zerlegen. Es gibt dabei wieder unterschiedliche herangehensweisen wie das mithilfe von Software passieren kann. Spacy verwendet dabei statistische Verfahren (Markov Models). So ist die wahrscheinlichkeit relativ hoch das nach einem "dem" ein Substantiv im Dativ folgt. Diese Wahrscheinlichkeitskalkulationen passieren jetzt über mehrere Wörter hinweg, was dann zu einem relativ guten Ergebnis führt.

In [64]:
doc = nlp('Der Arzt hilft dem Patienten.')
spacy_rendering = {"words": [], "arcs": []}
for token in doc:
    spacy_rendering["words"].append({"text": token.text, "tag": token.tag_})
HTML(displacy.render(spacy_rendering, style='dep', manual=True, options={"compact": True}))

Für POS tagging alleine gibt es nicht so viele Usecases, allerdings ist es die Grundlage für Dependency Parsing und Named Entity Recognition.

### Depedency Parsing
Beim dependency parsing geht es darum, herauszufinden welches Wort sich wie auf welches andere Wort bezieht und hilft so auch dabei Haupt- und Nebensätze zu separieren um die Kernaussage eines Satzes herauszufinden.

In [65]:
doc = nlp('Der Arzt hilft dem Patienten.')
HTML(displacy.render(doc, style='dep'))

### Named Entity Recognition (NER)


Beim NER geht es darum Informationen (insb. feststehende Begriffe) aus Texten zu extrahieren und zu klassifizieren. Dies passiert heute im Normalfall auf basis der annotierten Daten (POS; depedency tree) und 

In [86]:
doc = nlp('Wie wird das Wetter morgen in Hamburg?')
HTML(displacy.render(doc, style='ent'))


Statistische Methoden für NER ermöglichen uns auch für nicht im Trainingsset vorhandenen Namen/… Tags mit einer reltiv hohen Genauigkeit zu vergeben.

In [84]:
doc = nlp('Wer ist Alan Turing?')
HTML(displacy.render(doc, style='ent'))


In [85]:
doc = nlp('Wer ist Lilith Wittmann?')
HTML(displacy.render(doc, style='ent'))


# Bücher und andere Resourcen
- [**spacy Doku** ](https://spacy.io/usage/spacy-101)
- [nltk doku](https://www.nltk.org/)
- [Next Generation Natural Language Processing with Python - Alexis Rutherford](https://www.safaribooksonline.com/library/view/natural-language-processing/9781787285101/)
- [Natural Language Processing with Python Cookbook](https://www.safaribooksonline.com/library/view/natural-language-processing/9781787289321/)